In [2]:
import os
import time
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
from scipy.sparse import csr_matrix
import logging
#from skbio.stats.composition import clr

import matplotlib.pyplot as plt
import matplotlib.axes as axes
#For CLR of ADTs
import scipy
import scipy.stats
from sklearn.preprocessing import scale

BILLING_PROJECT_ID = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE = os.environ['WORKSPACE_NAME']
bucket = os.environ['WORKSPACE_BUCKET']

sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

2025-02-03 22:15:54.459360: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-03 22:16:18.373262: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-02-03 22:16:18.375333: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

scanpy==1.10.1 anndata==0.10.8 umap==0.5.6 numpy==1.24.4 scipy==1.13.0 pandas==2.0.3 scikit-learn==1.3.0 statsmodels==0.14.0 igraph==0.11.8 pynndescent==0.5.12


In [ ]:
#!gsutil cp $bucket/cellranger_output_directory/bladder/harmony/final_obj_without1132_high_resolution_annotated.h5ad .  

In [3]:
adata= sc.read_h5ad("harmonized_myeloid_only_annotated.h5ad")

In [4]:
TAM1df = adata[adata.obs['leiden'] == 'TAM PARP14+ SPP1+', :]
TAM1df.obs

,doublet_score,predicted_doublet,n_genes,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,total_counts_mito,log1p_total_counts_mito,pct_counts_mito,...,log1p_total_counts_ribo,pct_counts_ribo,total_counts_mt,pct_counts_mt,platelet,RBC,batch,leiden,response,seurat_clusters
CTGCTACTCAAGACTC-1-0,0.174171,False,517,533,6.280396,707.0,6.562444,0.0,0.000000,0.000000,...,1.791759,0.707214,0.0,0.000000,False,False,0,TAM PARP14+ SPP1+,Non-Responder,TAM PARP14+ SPP1+
GCTGACATCCGTAAAC-1-0,0.150491,False,334,351,5.863631,439.0,6.086775,10.0,2.397895,2.277904,...,1.386294,0.683371,10.0,2.277904,False,False,0,TAM PARP14+ SPP1+,Non-Responder,TAM PARP14+ SPP1+
CAAACTGGTCCGTAGG-1-0,0.212946,False,404,427,6.059123,471.0,6.156979,2.0,1.098612,0.424628,...,1.791759,1.061571,2.0,0.424628,False,False,0,TAM PARP14+ SPP1+,Non-Responder,TAM M1 HLA-DPB1+
GAAGTGATCACCTGTC-1-0,0.187616,False,265,286,5.659482,324.0,5.783825,14.0,2.708050,4.320988,...,0.693147,0.308642,14.0,4.320988,False,False,0,TAM PARP14+ SPP1+,Non-Responder,TAM M1 HLA-DPB1+
GTTGCATAGAACAAGT-1-0,0.140010,False,248,265,5.583496,316.0,5.758902,3.0,1.386294,0.949367,...,1.386294,0.949367,3.0,0.949367,False,False,0,TAM PARP14+ SPP1+,Non-Responder,TAM PARP14+ SPP1+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACGAATCTCCCTCACG-1-13,0.121429,False,399,409,6.016157,512.0,6.240276,8.0,2.197225,1.562500,...,0.693147,0.195312,8.0,1.562500,False,False,13,TAM PARP14+ SPP1+,Responder,TAM PARP14+ SPP1+
ACGACAAAGTAGCGGG-1-13,0.073078,False,368,375,5.929589,449.0,6.109248,2.0,1.098612,0.445434,...,1.945910,1.336303,2.0,0.445434,False,False,13,TAM PARP14+ SPP1+,Responder,TAM PARP14+ SPP1+
AAACATGCATGAATCT-1-13,0.064354,False,279,285,5.655992,321.0,5.774551,4.0,1.609438,1.246106,...,1.098612,0.623053,4.0,1.246106,False,False,13,TAM PARP14+ SPP1+,Responder,TAM PARP14+ SPP1+
CGCTGTGCACCAGCAT-1-13,0.149040,False,269,281,5.641907,325.0,5.786897,8.0,2.197225,2.461539,...,1.386294,0.923077,8.0,2.461539,False,False,13,TAM PARP14+ SPP1+,Responder,TAM PARP14+ SPP1+


In [5]:
TAM1df.obs['response'].value_counts(sort=False)

response
Non-Responder    2036
Responder         245
Name: count, dtype: int64

In [12]:
rdf = pd.DataFrame(adata[adata.obs['leiden'] == 'TAM PARP14+ SPP1+', :].obs['batch'].value_counts(sort=False))
rdf

,count
batch,
0,5
1,904
2,198
3,368
4,362
5,84
6,115
8,174
9,30


In [8]:
rdf.sum()

count    2281
dtype: int64

Non-Responder    2036

Responder         245

In [25]:
rdf.rename(columns={'count': 'spp1+ count'}, inplace=True)

In [24]:
df = pd.DataFrame(adata.obs['batch'].value_counts(sort=False))
df['response']=['NR','NR','NR','NR','NR','NR','NR','NR','R','R','R','R','R','R']
df

,count,response
batch,,
0,21,NR
1,964,NR
2,312,NR
3,447,NR
4,407,NR
5,132,NR
6,353,NR
7,3,NR
8,576,R


In [46]:
df_merged = df.merge(rdf, on='batch', how='outer')
df_merged.fillna(0, inplace=True)
df_merged['spp1+ count/all myeloid count'] = df_merged['spp1+ count']/df_merged['count']

In [49]:
df_merged

,count,response,spp1+ count,spp1+ count/all myeloid count
batch,,,,
0,21,NR,5.0,0.238095
1,964,NR,904.0,0.937759
2,312,NR,198.0,0.634615
3,447,NR,368.0,0.823266
4,407,NR,362.0,0.889435
5,132,NR,84.0,0.636364
6,353,NR,115.0,0.325779
7,3,NR,0.0,0.000000
8,576,R,174.0,0.302083


In [51]:
# Create two independent samples

groupR = df_merged[df_merged['response']=='R']['spp1+ count/all myeloid count']
groupNR = df_merged[df_merged['response']=='NR']['spp1+ count/all myeloid count']


# Perform the t-test
t_stat, p_value = stats.ttest_ind(groupR, groupNR)

print("t-statistic:", t_stat)
print("p-value:", p_value)

t-statistic: -2.624466215023132
p-value: 0.02220122464041287


#p-value: 0.02220122464041287